In [1]:
from ase import io
from ase.units import mol, kcal, kJ, Hartree
import pandas as pd
from cluster_scripts import *
import pyblock

dft_functionals = ['01_PBE-D2-Ne','02_revPBE-D4','03_vdW-DF','04_rev-vdW-DF2','05_PBE0-D4','06_B3LYP-D2-Ne']

kcalmol_to_meV = kcal/mol * 1000
kjmol_to_meV = kJ/mol * 1000
mha_to_meV = Hartree

In [25]:
eads_bond_length = {"CCSD(T)_small": [],
                    "MP2_small": [],
                    "MP2_large": [],
                    "Final": [],
                    "01_PBE-D2-Ne": [],
                    "02_revPBE-D4": [],
                    "03_vdW-DF": [],
                    "04_B3LYP-D2-Ne": []}

r_list = ['2.36','2.38','2.40','2.41','2.42','2.43','2.44','2.45','2.46','2.47','2.48','2.49','2.50','2.51','2.52','2.53','2.54','2.55','2.56','2.57','2.58','2.60','2.62']


for j in r_list:
    ene_hf_small = []
    ene_hf_large = []
    ene_mp2_small = []
    ene_mp2_large = []
    ene_cc_small = []
    for k in ['TZ','QZ']:
        ene_hf_small += [get_eads('Data/Bond_Dist/{0}/CC/{1}'.format(j,k),code_format='mrcc',typ='hf')]
        ene_mp2_small += [get_eads('Data/Bond_Dist/{0}/CC/{1}'.format(j,k),code_format='mrcc',typ='mp2')]
        ene_cc_small += [get_eads('Data/Bond_Dist/{0}/CC/{1}'.format(j,k),code_format='mrcc',typ='ccsdt')]
        ene_hf_large += [get_eads('Data/Bond_Dist/{0}/MP2/{1}'.format(j,k),code_format='mrcc',typ='hf')]
        ene_mp2_large += [get_eads('Data/Bond_Dist/{0}/MP2/{1}'.format(j,k),code_format='mrcc',typ='lmp2_corr')]

    
    eads_bond_length['CCSD(T)_small'] += [(extrapolate.get_cbs(ene_hf_small[0],ene_cc_small[0],ene_hf_small[1],ene_cc_small[1],X=3,Y=4,family='mixcc',output=False))[-1]*Hartree*1000]
    eads_bond_length['MP2_small'] += [(extrapolate.get_cbs(ene_hf_small[0],ene_mp2_small[0],ene_hf_small[1],ene_mp2_small[1],X=3,Y=4,family='mixcc',output=False))[-1]*Hartree*1000]
    eads_bond_length['MP2_large'] += [(extrapolate.get_cbs(ene_hf_large[0],ene_mp2_large[0],ene_hf_large[1],ene_mp2_large[1],X=3,Y=4,family='mixcc',output=False))[-1]*Hartree*1000]
    eads_bond_length['Final'] += [eads_bond_length['MP2_large'][-1] + eads_bond_length['CCSD(T)_small'][-1] - eads_bond_length['MP2_small'][-1]]

# Getting the polyfit stuff for the Final CC calculations
# min_idx = np.argmin(eads_bond_length['Final'])
# a = np.polyfit([float(x) for x in r_list[min_idx-5: min_idx + 5]], eads_bond_length['Final'][min_idx-5: min_idx + 5],deg=2)

# print(-a[1]/(2*a[0]))
mg_c_bond_length = {
    "CCSD(T)_small": [0,0,0],
    "MP2_small": [0,0,0],
    "MP2_large": [0,0,0],
    "Final": [0,0,0],
    "01_PBE-D2-Ne": [0,0,0],
    "02_revPBE-D4": [0,0,0],
    "03_vdW-DF": [0,0,0],
    "04_B3LYP-D2-Ne": [0,0,0]}

for i in ['MP2_small','MP2_large','CCSD(T)_small','Final']: #"01_PBE-D2-Ne", "02_revPBE-D4","03_vdW-DF", "04_B3LYP-D2-Ne"]:
    min_idx = np.argmin(eads_bond_length[i])
    a = np.polyfit([float(x) for x in r_list[min_idx-5: min_idx + 5]], eads_bond_length[i][min_idx-5: min_idx + 5],deg=2)
    b = np.polyfit([float(x) for x in r_list], eads_bond_length[i],deg=4)

    # a = np.polyfit([float(x) for x in r_list], eads_bond_length[i],deg=2)
    mg_c_bond_length[i][0] = -a[1]/(2*a[0])
    mg_c_bond_length[i][1] = a[0] * mg_c_bond_length[i][0]**2 + a[1] * mg_c_bond_length[i][0] + a[2]
    mg_c_bond_length[i][2] = a[0] * (mg_c_bond_length[i][0]-0.1)**2 + a[1] * (mg_c_bond_length[i][0]-0.1) + a[2] - mg_c_bond_length[i][1]

    print(np.polyval(b,2.441), mg_c_bond_length[i][1])


-153.44403274678916 -157.69614264814754
-191.7362553425337 -193.97802751408426
-160.42450992269733 -165.29462996190796
-198.71673251841275 -201.40688690703064


In [16]:
mg_c_bond_length

{'Final': [2.5086752819101634, -201.40688690703064, 5.460044526779257],
 '01_PBE-D2-Ne': [0, 0, 0],
 '02_revPBE-D4': [0, 0, 0],
 '03_vdW-DF': [0, 0, 0],
 '04_B3LYP-D2-Ne': [0, 0, 0]}

In [18]:
eads_bond_length

{'CCSD(T)_small': [-145.0964047094569,
  -150.12065916366285,
  -154.275467638076,
  -156.05346011231137,
  -157.645153995469,
  -159.05995208685164,
  -160.30687367687597,
  -161.39453841556124,
  -162.33117856264778,
  -163.12467361204065,
  -163.78254743749795,
  -164.31197732855685,
  -164.71983614910843,
  -165.01268666099267,
  -165.19680550486075,
  -165.27669102649503,
  -165.26100971109952,
  -165.15374089265578,
  -164.96007854189523,
  -164.68500156927408,
  -164.3350895392235,
  -163.41943972779387,
  -162.24877265959407],
 'MP2_small': [-138.86424336147812,
  -143.6856130465435,
  -147.6495783518824,
  -149.33658554458452,
  -150.8402266525415,
  -152.16986362422648,
  -153.33448705014064,
  -154.3426871155207,
  -155.20266518978883,
  -155.9222694743946,
  -156.50899021081275,
  -156.9699748336737,
  -157.31206191185234,
  -157.54177926096642,
  -157.665376732085,
  -157.68759587213378,
  -157.6165109848198,
  -157.45634930696542,
  -157.2122684642147,
  -156.889211463699

In [57]:
ene_dmc_list = {x: {y: 0 for y in ['01-1E-1','01-1E-1_2std','02-3E-2','02-3E-2_2std','03-1E-2','03-1E-2_2std']} for x in ['4L','2L', '2L_He']}

# Getting the 4L and 2L first at 0.1 timestep

data = []
for i in [1,2,3,4]:
    a = np.loadtxt('Data/DMC/{0}/01-1E-1/dmc_run_01.hist'.format(i))
    b = np.loadtxt('Data/DMC/{0}/01-1E-1/dmc_run_02.hist'.format(i))
    dummy_data = np.append(a[3000:,3],b[3000:,3])
    if i == 1 or i == 2:
        c = np.loadtxt('Data/DMC/{0}/01-1E-1/dmc_run_03.hist'.format(i))
        dummy_data = np.append(dummy_data,c[3000:,3])
    rand_data = pd.Series(dummy_data*Hartree*1000)
    (data_length, reblock_data, covariance) = pyblock.pd_utils.reblock(rand_data)
    pyblock.pd_utils.reblock_summary(reblock_data)
    data += [pyblock.pd_utils.reblock_summary(reblock_data)]        

ene_dmc_list['4L']['01-1E-1'] = pyblock.error.subtraction(data[0],data[1],0,1)['mean'][0]
ene_dmc_list['4L']['01-1E-1_2std'] = 2*pyblock.error.subtraction(data[0],data[1],0,1)['standard error'][0]
ene_dmc_list['2L']['01-1E-1'] = pyblock.error.subtraction(data[2],data[3],0,1)['mean'][0]
ene_dmc_list['2L']['01-1E-1_2std'] = 2*pyblock.error.subtraction(data[2],data[3],0,1)['standard error'][0]

# Getting the 4L at 0.03 timestep

data = []

for i in [1,2]: #,3,4]:
    a = np.loadtxt('Data/DMC/{0}/02-3E-2/dmc_run_01.hist'.format(i))
    b = np.loadtxt('Data/DMC/{0}/02-3E-2/dmc_run_02.hist'.format(i))
    c = np.loadtxt('Data/DMC/{0}/02-3E-2/dmc_run_03.hist'.format(i))
    dummy_data = np.append(a[3000:,3],b[3000:,3])
    dummy_data = np.append(dummy_data,c[3000:,3])
    if i == 1:
        d = np.loadtxt('Data/DMC/{0}/02-3E-2/dmc_run_04.hist'.format(i))
        e = np.loadtxt('Data/DMC/{0}/02-3E-2/dmc_run_05.hist'.format(i))
        dummy_data = np.append(dummy_data,d[3000:,3])
        dummy_data = np.append(dummy_data,e[3000:,3])

    rand_data = pd.Series(dummy_data*Hartree*1000)
    (data_length, reblock_data, covariance) = pyblock.pd_utils.reblock(rand_data)
    pyblock.pd_utils.reblock_summary(reblock_data)
    data += [pyblock.pd_utils.reblock_summary(reblock_data)]

ene_dmc_list['4L']['02-3E-2'] = pyblock.error.subtraction(data[0],data[1],0,1)['mean'][0]
ene_dmc_list['4L']['02-3E-2_2std'] = 2*pyblock.error.subtraction(data[0],data[1],0,1)['standard error'][0]

# Getting the 2L_He at 0.1, 0.03 and 0.01 timestep

for j in ['01-1E-1','02-3E-2','03-1E-2']:

    data = []

    for i in [3,4]: #,3,4]:
        a = np.loadtxt('Data/DMC/{0}_He/{1}/dmc.hist'.format(i,j))
        dummy_data = a[3000:,3]
        rand_data = pd.Series(dummy_data*Hartree*1000)
        (data_length, reblock_data, covariance) = pyblock.pd_utils.reblock(rand_data)
        pyblock.pd_utils.reblock_summary(reblock_data)
        data += [pyblock.pd_utils.reblock_summary(reblock_data)]

    ene_dmc_list['2L_He'][j] = pyblock.error.subtraction(data[0],data[1],0,1)['mean'][0]
    ene_dmc_list['2L_He'][j + '_2std'] = 2*pyblock.error.subtraction(data[0],data[1],0,1)['standard error'][0]



In [62]:
df = pd.DataFrame(ene_dmc_list)
df1 = df[['4L','2L_He']].T
df1 = df1.round().astype(int)
df1.to_clipboard( index=True, excel=True,sep=',')


In [54]:
# Finite size error correction


data = []

for i in [1,2]:
    if i == 2:
        a = np.loadtxt('Data/DMC/{0}_MPC/01-1E-1/dmc.hist'.format(i))
        dummy_data = a[300:,4] - a[300:,3]
    elif i == 1:
        a1 = np.loadtxt('Data/DMC/{0}_MPC/01-1E-1/dmc_run_01.hist'.format(i))
        a2 = np.loadtxt('Data/DMC/{0}_MPC/01-1E-1/dmc_run_02.hist'.format(i))
        a3 = np.loadtxt('Data/DMC/{0}_MPC/01-1E-1/dmc_run_03.hist'.format(i))
        dummy_data = np.append(a1[300:,4] - a1[300:,3],a2[300:,4] - a2[300:,3])
        dummy_data = np.append(dummy_data,a3[300:,4] - a3[300:,3])

    rand_data = pd.Series(dummy_data*Hartree*1000)
    (data_length, reblock_data, covariance) = pyblock.pd_utils.reblock(rand_data)
    pyblock.pd_utils.reblock_summary(reblock_data)
    data += [pyblock.pd_utils.reblock_summary(reblock_data)]

ene_2l = [ene_dmc_list['2L_He']['02-3E-2'], ene_dmc_list['2L_He']['02-3E-2_2std']]
delta_fse = [pyblock.error.subtraction(data[0],data[1],0,1)['mean'][0],2*pyblock.error.subtraction(data[0],data[1],0,1)['standard error'][0]]
delta_2l_4l = [ene_dmc_list['4L']['01-1E-1'] - ene_dmc_list['2L']['01-1E-1'], np.sqrt(ene_dmc_list['4L']['01-1E-1_2std']**2 + ene_dmc_list['2L']['01-1E-1_2std']**2)]
delta_ipfse = [3,0]
print(ene_2l,delta_fse,delta_2l_4l,delta_ipfse)

[-159.3021583855152, 6.824266463160695] [-33.1784023574055, 4.84462637209061] [-8.91541859600693, 9.573701248873194] [3, 0]


-142.4241454321891

Experimental and Theoretical Data

In [7]:

# Experiments go as [hads, experiment_type,year]
experimental_hads = {
    'Furuyama et al.': [3.8*kcalmol_to_meV,'Isothermal adsorption',1978],
    'Paukshtis et al.': [15*kjmol_to_meV,'IR spectroscopy',1981],
    'Henry et al.': [420,'Auger spectroscopy',1991],
    'He et al.': [9.9*kcalmol_to_meV,'Isothermal adsorption',1992],
    'Wichtendahl et al.': [140,'Thermal programmed desorption',1999],
    'Dohnalek et al.': [20*kjmol_to_meV,'Thermal programmed desorption',2001],
    'Spoto et al.1': [11*kjmol_to_meV,'FTIR spectroscopy',2003],
    'Spoto et al.2': [12.5*kjmol_to_meV,'FTIR spectroscopy',2004],
    'Sterrer et al.': [15*kjmol_to_meV,'Thermal programmed desorption',2006]
}



# Computation goes as [eads,]
computational_eads = {
    'Ugliengo et al.': [12.7*kjmol_to_meV,'cluster MP2:B3LYP',2002,'HL:LL mechanical embedding'],
    'Herschend et al.': [124,'cluster MP2',2006,'Electrostatic embedding'],
    'Qin et al.': [110,'cluster CISD',2008,'Electrostatic embedding'],
    'Staemmler': [124,'cluster CEPA',2011,'Method of local increments'],
    'Boese et al.': [21*kjmol_to_meV,'cluster MP2+deltaCC:PBE-D2',2013,'HL:LL mechanical embedding'],
    'Li et al.': [3*kjmol_to_meV,'cluster MP2',2015,'Gas phase cluster-in-molecule approach'],
    'Bajdich et al.': [310,'periodic RPA',2015,'Supercell approach'],
    'Heuser et al.': [ 15.0*mha_to_meV, 'cluster CC2',2016,'Wave-function frozen-density embedding'],
    'Mazheika and Levchenko': [-70,'cluster CCSD(T)',2016,'Electrostatic embedding'],
    'Alessio et al.': [20.8*kjmol_to_meV,'cluster MP2+deltaCC:PBE-D2',2019,'HL:LL mechanical embedding'],
    'Mitra et al.': [9.18*kcalmol_to_meV,'periodic CCSD',2022,'Supercell approach']
}

thermal_correction = 28

# 1 kjmol for Boese and 0.5 kjmol error for Alessio

In [4]:
df = pd.DataFrame(experimental_hads).T

NameError: name 'experimental_hads' is not defined

In [13]:
df = pd.DataFrame(experimental_hads).T
df = df.reset_index()
df.columns = ['Reference','Hads (meV)','Method','Year']
df['Hads (meV)'] = df['Hads (meV)'].apply(lambda x: -round(x))

eads_list = []
for i in experimental_hads:
    if 'desorption' in experimental_hads[i][1]:
        eads_list += [experimental_hads[i][0] + thermal_correction + 2.5]
    else:
        eads_list += [experimental_hads[i][0] + thermal_correction]
df['Eads (meV)'] = eads_list
df['Eads (meV)'] = df['Eads (meV)'].round().astype(int)

# df['Hads (meV)'] - thermal_correction
# df['Details'] = pd.Series(dtype='str')
df = df[['Reference','Eads (meV)','Hads (meV)','Method','Year']]
print(df)

df1 = pd.DataFrame(computational_eads).T
df1 = df1.reset_index()
df1.columns = ['Reference','Eads (meV)','Method','Year','Details']
df1['Eads (meV)'] = df1['Eads (meV)'].apply(lambda x: -round(x))
df1['Hads (meV)'] = df1['Eads (meV)'] + thermal_correction
df1 = df1[['Reference','Eads (meV)','Hads (meV)','Method','Year','Details']]
df1

df_final = pd.concat([df,df1])
df_final
df_final.to_clipboard( index=False, excel=True,sep=',')


            Reference  Eads (meV)  Hads (meV)                         Method  \
0     Furuyama et al.         193        -165          Isothermal adsorption   
1    Paukshtis et al.         183        -155                IR spectroscopy   
2        Henry et al.         448        -420             Auger spectroscopy   
3           He et al.         457        -429          Isothermal adsorption   
4  Wichtendahl et al.         170        -140  Thermal programmed desorption   
5     Dohnalek et al.         238        -207  Thermal programmed desorption   
6       Spoto et al.1         142        -114              FTIR spectroscopy   
7       Spoto et al.2         158        -130              FTIR spectroscopy   
8      Sterrer et al.         186        -155  Thermal programmed desorption   

   Year  
0  1978  
1  1981  
2  1991  
3  1992  
4  1999  
5  2001  
6  2003  
7  2004  
8  2006  


In [43]:
dft_functionals = ['01_PBE-D2-Ne','02_revPBE-D4','03_vdW-DF','04_rev-vdW-DF2','05_PBE0-D4','06_B3LYP-D2-Ne']

ene_ads_list = {'Eads_True': {x:0 for x in dft_functionals},
                'Eads_Approx':{x:0 for x in dft_functionals}}

for i in ['02_revPBE-D4']:
    ad_slab = find_energy('Data/DFT/Eads/{0}/AD_SLAB/OUTCAR'.format(i),code_format='vasp')
    ad =  find_energy('Data/DFT/Eads/{0}/AD/OUTCAR'.format(i),code_format='vasp')
    slab = find_energy('Data/DFT/Eads/{0}/SLAB/OUTCAR'.format(i),code_format='vasp')
    ad_fs = find_energy('Data/DFT/Eint/{0}/SLAB_FS/OUTCAR'.format(i),code_format='vasp')
    slab_fs = find_energy('Data/DFT/Eint/{0}/AD_FS/OUTCAR'.format(i),code_format='vasp')
    ad_slab_fs = find_energy('Data/DFT/Eint/{0}/AD_SLAB/OUTCAR'.format(i),code_format='vasp')
    e_relax = (slab_fs-slab) + (ad_fs - ad)


squared_sum = 0
for i in dft_functionals:
    ad_slab = find_energy('Data/DFT/Eads/{0}/AD_SLAB/OUTCAR'.format(i),code_format='vasp')
    ad =  find_energy('Data/DFT/Eads/{0}/AD/OUTCAR'.format(i),code_format='vasp')
    slab = find_energy('Data/DFT/Eads/{0}/SLAB/OUTCAR'.format(i),code_format='vasp')
    ad_fs = find_energy('Data/DFT/Eint/{0}/SLAB_FS/OUTCAR'.format(i),code_format='vasp')
    slab_fs = find_energy('Data/DFT/Eint/{0}/AD_FS/OUTCAR'.format(i),code_format='vasp')
    ad_slab_fs = find_energy('Data/DFT/Eint/{0}/AD_SLAB/OUTCAR'.format(i),code_format='vasp')
    eads = (ad_slab-ad-slab)*1000
    eint = (ad_slab_fs-slab_fs-ad_fs)*1000
    eads_pred = (ad_slab_fs-slab_fs-ad_fs + e_relax)*1000
    ene_ads_list['Eads_True'][i] = eads
    ene_ads_list['Eads_Approx'][i] = eads_pred
   
    # print(ad_slab)
    squared_sum += (eads-eads_pred)**2


df = pd.DataFrame(ene_ads_list)
df['Difference'] = df['Eads_True'] - df['Eads_Approx']
df = df.round().astype(int)
print(df)
print('RMSD:',round(np.sqrt(squared_sum/5)))

df.to_clipboard( index=True, excel=True,sep=',')


# eads = 
# eint = 

                Eads_True  Eads_Approx  Difference
01_PBE-D2-Ne         -228         -225          -4
02_revPBE-D4         -207         -207           0
03_vdW-DF            -232         -212         -20
04_rev-vdW-DF2       -266         -265          -1
05_PBE0-D4           -234         -241           7
06_B3LYP-D2-Ne       -149         -148          -1
RMSD: 10


In [46]:
# Reading in the lattice parameter
latpar_functional = []

for i in dft_functionals:
    a = io.read('Data/DFT/Unit_Cell/{0}/CONTCAR'.format(i))
    latpar_functional += [a.get_cell()[0][0]]

# print(latpar_functional)

bond_length = []

for i in dft_functionals:
    a = io.read('Data/DFT/Eads/{0}/AD_SLAB/CONTCAR'.format(i))
    if i == '02_revPBE-D4' or i == '03_vdW-DF':
        bond_length += [np.linalg.norm(a.get_positions()[0] - a.get_positions()[32])]
    else:
        bond_length += [np.linalg.norm(a.get_positions()[0] - a.get_positions()[17])]

# print(bond_length)

df['Lattice Parameter'] = [round(x,3) for x in latpar_functional]
df['Bond length'] = [round(x,3) for x in bond_length]
df1 = df.drop(['Eads_Approx','Difference'],axis=1)
new_row = {'Eads_True':-211, 'Lattice Parameter':4.212, 'Bond length': 2.508}
df2 = df1.append(pd.DataFrame([new_row], index=['Reference'],columns=df1.columns))

df2.to_clipboard( index=True, excel=True,sep=',')


In [55]:
# Convergence of the MP2 adsorption energies
lot_list = ['MP2_DZ','MP2_TZ','MP2_QZ','MP2_TZQZ','MP2_DZTZ','MP2_DZTZ_lc','MP2_DZTZ_lc_NOCP','LCCSDT_TZQZ','LMP2_TZQZ','CCCSDT_TZQZ','CMP2_TZQZ']
ene_list = {y: [] for y in lot_list } 
ene_extrap_list = {y: [] for y in lot_list } 
tot_atom_list = [6, 22, 34, 42, 58, 82, 84, 100, 108]

for j in range(1,9):
    ene_list['MP2_DZ'] += [get_eads('Data/MP2_Convergence/{0}/DZ'.format(j), code_format='mrcc',typ='lmp2_tot')*Hartree*1000]
    if j < 3:
        ene_extrap_list['MP2_DZ'] += [0]
    else:
        gamma_best = find_co_gamma(ene_list['MP2_DZ'])
        slope, emp2_infty, r, p, se = linregress([1/(x**(gamma_best)) for x in tot_atom_list[:len(ene_list['MP2_DZ'])]],\
            [float(x) for x in ene_list['MP2_DZ']])
        ene_extrap_list['MP2_DZ'] += [emp2_infty]

for j in range(1,6):
    ene_hf = []
    ene_mp2 = []
    for k in ['TZ','QZ']:
        ene_list['MP2_'+k] += [get_eads('Data/MP2_Convergence/{0}/{1}'.format(j,k), code_format='mrcc',typ='lmp2_tot')*Hartree*1000]
        ene_hf += [get_eads('Data/MP2_Convergence/{0}/{1}'.format(j,k), code_format='mrcc',typ='hf')*Hartree*1000]
        ene_mp2 += [get_eads('Data/MP2_Convergence/{0}/{1}'.format(j,k), code_format='mrcc',typ='lmp2_corr')*Hartree*1000]
    dummy = extrapolate.get_cbs(ene_hf[0],ene_mp2[0],ene_hf[1],ene_mp2[1],X=3,Y=4,family='mixcc',output=False)
    ene_list['MP2_TZQZ'] += [dummy[-1]]

    if j < 3:
        ene_extrap_list['MP2_TZQZ'] += [0]
    else:
        gamma_best = find_co_gamma(ene_list['MP2_TZQZ'])
        slope, emp2_infty, r, p, se = linregress([1/(x**(gamma_best)) for x in tot_atom_list[:len(ene_list['MP2_TZQZ'])]],\
            [float(x) for x in ene_list['MP2_TZQZ']])
        ene_extrap_list['MP2_TZQZ'] += [emp2_infty]


for j in range(1,6):
    ene_hf = []
    ene_mp2 = []
    for k in ['DZ','TZ']:
        ene_hf += [get_eads('Data/MP2_Convergence/{0}/{1}_lc'.format(j,k), code_format='mrcc',typ='hf')*Hartree*1000]
        ene_mp2 += [get_eads('Data/MP2_Convergence/{0}/{1}_lc'.format(j,k), code_format='mrcc',typ='lmp2_corr')*Hartree*1000]
    dummy = extrapolate.get_cbs(ene_hf[0],ene_mp2[0],ene_hf[1],ene_mp2[1],X=2,Y=3,family='mixcc',output=False)
    ene_list['MP2_DZTZ_lc'] += [dummy[-1]]

    if j < 3:
        ene_extrap_list['MP2_DZTZ_lc'] += [0]
    else:
        gamma_best = find_co_gamma(ene_list['MP2_DZTZ_lc'])
        slope, emp2_infty, r, p, se = linregress([1/(x**(gamma_best)) for x in tot_atom_list[:len(ene_list['MP2_DZTZ_lc'])]],\
            [float(x) for x in ene_list['MP2_DZTZ_lc']])
        ene_extrap_list['MP2_DZTZ_lc'] += [emp2_infty]

for j in range(1,6):
    ene_hf = []
    ene_mp2 = []
    for k in ['DZ','TZ']:
        ene_hf += [get_eads('Data/MP2_Convergence/{0}/{1}_lc'.format(j,k), code_format='mrcc',typ='hf',structs=['AD_SLAB','SLAB_NOCP','AD_NOCP'])*Hartree*1000]
        ene_mp2 += [get_eads('Data/MP2_Convergence/{0}/{1}_lc'.format(j,k), code_format='mrcc',typ='lmp2_corr',structs=['AD_SLAB','SLAB_NOCP','AD_NOCP'])*Hartree*1000]
    dummy = extrapolate.get_cbs(ene_hf[0],ene_mp2[0],ene_hf[1],ene_mp2[1],X=2,Y=3,family='mixcc',output=False)
    ene_list['MP2_DZTZ_lc_NOCP'] += [dummy[-1]]
    # print(ene_hf,ene_mp2)

    if j < 3:
        ene_extrap_list['MP2_DZTZ_lc_NOCP'] += [0]
    else:
        gamma_best = find_co_gamma(ene_list['MP2_DZTZ_lc_NOCP'])
        slope, emp2_infty, r, p, se = linregress([1/(x**(gamma_best)) for x in tot_atom_list[:len(ene_list['MP2_DZTZ_lc_NOCP'])]],[float(x) for x in ene_list['MP2_DZTZ_lc_NOCP']])
        ene_extrap_list['MP2_DZTZ_lc_NOCP'] += [emp2_infty]

for j in range(1,6):
    ene_hf = []
    ene_mp2 = []
    for k in ['DZ','TZ']:
        ene_hf += [get_eads('Data/MP2_Convergence/{0}/{1}'.format(j,k), code_format='mrcc',typ='hf')*Hartree*1000]
        ene_mp2 += [get_eads('Data/MP2_Convergence/{0}/{1}'.format(j,k), code_format='mrcc',typ='lmp2_corr')*Hartree*1000]
    dummy = extrapolate.get_cbs(ene_hf[0],ene_mp2[0],ene_hf[1],ene_mp2[1],X=2,Y=3,family='mixcc',output=False)
    ene_list['MP2_DZTZ'] += [dummy[-1]]

    if j < 3:
        ene_extrap_list['MP2_DZTZ'] += [0]
    else:
        gamma_best = find_co_gamma(ene_list['MP2_DZTZ'])
        slope, emp2_infty, r, p, se = linregress([1/(x**(gamma_best)) for x in tot_atom_list[:len(ene_list['MP2_DZTZ'])]],\
            [float(x) for x in ene_list['MP2_DZTZ']])
        ene_extrap_list['MP2_DZTZ'] += [emp2_infty]


for j in range(1,4):
    ene_hf = []
    ene_mp2 = []
    for k in ['DZ','TZ']:
        # ene_list['LCCSD_'+k] += [get_eads('Data/CC_Convergence/{0}/{1}'.format(j,k), code_format='mrcc',typ='lmp2_tot')*Hartree*1000]
        ene_hf += [get_eads('Data/CC_Convergence/{0}_Local/{1}'.format(j,k), code_format='mrcc',typ='hf')*Hartree*1000]
        ene_mp2 += [get_eads('Data/CC_Convergence/{0}_Local/{1}'.format(j,k), code_format='mrcc',typ='lccsdt')*Hartree*1000]
    dummy = extrapolate.get_cbs(ene_hf[0],ene_mp2[0],ene_hf[1],ene_mp2[1],X=2,Y=3,family='mixcc',output=False)
    ene_list['LCCSDT_TZQZ'] += [dummy[-1]]
    ene_hf = []
    ene_mp2 = []
    for k in ['DZ','TZ']:
        # ene_list['LCCSD_'+k] += [get_eads('Data/CC_Convergence/{0}/{1}'.format(j,k), code_format='mrcc',typ='lmp2_tot')*Hartree*1000]
        ene_hf += [get_eads('Data/CC_Convergence/{0}_Local/{1}'.format(j,k), code_format='mrcc',typ='hf')*Hartree*1000]
        ene_mp2 += [get_eads('Data/CC_Convergence/{0}_Local/{1}'.format(j,k), code_format='mrcc',typ='lmp2')*Hartree*1000]
    dummy = extrapolate.get_cbs(ene_hf[0],ene_mp2[0],ene_hf[1],ene_mp2[1],X=2,Y=3,family='mixcc',output=False)
    ene_list['LMP2_TZQZ'] += [dummy[-1]]

for j in range(1,2):
    ene_hf = []
    ene_mp2 = []
    for k in ['TZ','QZ']:
        ene_hf += [get_eads('Data/CC_Convergence/{0}_Canonical/{1}'.format(j,k), code_format='mrcc',typ='hf')*Hartree*1000]
        ene_mp2 += [get_eads('Data/CC_Convergence/{0}_Canonical/{1}'.format(j,k), code_format='mrcc',typ='ccsdt')*Hartree*1000]
    dummy = extrapolate.get_cbs(ene_hf[0],ene_mp2[0],ene_hf[1],ene_mp2[1],X=3,Y=4,family='mixcc',output=False)
    ene_list['CCCSDT_TZQZ'] += [dummy[-1]]
    ene_hf = []
    ene_mp2 = []
    for k in ['TZ','QZ']:
        ene_hf += [get_eads('Data/CC_Convergence/{0}_Canonical/{1}'.format(j,k), code_format='mrcc',typ='hf')*Hartree*1000]
        ene_mp2 += [get_eads('Data/CC_Convergence/{0}_Canonical/{1}'.format(j,k), code_format='mrcc',typ='mp2')*Hartree*1000]
    dummy = extrapolate.get_cbs(ene_hf[0],ene_mp2[0],ene_hf[1],ene_mp2[1],X=3,Y=4,family='mixcc',output=False)
    ene_list['CMP2_TZQZ'] += [dummy[-1]]


In [59]:
trunc_ene_list = {}
for i in ['MP2_DZ','MP2_TZ','MP2_QZ','MP2_TZQZ','LMP2_TZQZ','LCCSDT_TZQZ','CCCSDT_TZQZ']:
    if len(ene_list[i]) < 8:
        dff_len = 8 - len(ene_list[i])
        trunc_ene_list[i] = ene_list[i] + [0]*dff_len

    else:
        trunc_ene_list[i] = ene_list[i]

df = pd.DataFrame(trunc_ene_list)
df.columns = ['MP2 DZ','MP2 TZ','MP2 QZ','MP2 CBS(TZ/QZ)','LMP2 CBS(TZ/QZ)','LNO-CCSD(T) CBS(TZ/QZ)','CCSD(T) CBS(TZ/QZ)']
df['# of atoms'] = tot_atom_list[:8]
df =df[['# of atoms','MP2 DZ','MP2 TZ','MP2 QZ','MP2 CBS(TZ/QZ)','LMP2 CBS(TZ/QZ)','LNO-CCSD(T) CBS(TZ/QZ)','CCSD(T) CBS(TZ/QZ)']]
df = df.round().astype(int)
df.index += 1
# df.to_clipboard(index=True, excel=True,sep=',')
# df

trunc_ene_extrap_list = {}
for i in ['MP2_DZ','MP2_DZ_Error','MP2_TZQZ']:

    if i == 'MP2_DZ_Error':
        trunc_ene_extrap_list[i] = [abs(x-ene_extrap_list['MP2_DZ'][-1]) for x in ene_extrap_list['MP2_DZ']][2:]
        continue


    if len(ene_extrap_list[i]) < 8:
        dff_len = 8 - len(ene_extrap_list[i])
        trunc_ene_extrap_list[i] = (ene_extrap_list[i] + [0]*dff_len)[2:]

    else:
        trunc_ene_extrap_list[i] = ene_extrap_list[i][2:]

df = pd.DataFrame(trunc_ene_extrap_list)
df.columns = ['MP2 DZ','MP2_DZ_Error','MP2 CBS(TZ/QZ)']
df = df.round().astype(int)
df.index += 3
df.to_clipboard(index=True, excel=True,sep=',')

df


trunc_ene_cc_list = {}
for i in ['LMP2_TZQZ','LCCSDT_TZQZ','CMP2_TZQZ','CCCSDT_TZQZ']:

    if len(ene_list[i]) < 3:
        dff_len = 3 - len(ene_list[i])
        trunc_ene_cc_list[i] = (ene_list[i] + [0]*dff_len)

    else:
        trunc_ene_cc_list[i] = ene_list[i]

df = pd.DataFrame(trunc_ene_cc_list)

df['deltacc_local'] = df['LCCSDT_TZQZ'] - df['LMP2_TZQZ']
df['deltacc'] = df['CCCSDT_TZQZ'] - df['CMP2_TZQZ']
df = df[['LMP2_TZQZ','LCCSDT_TZQZ','deltacc_local','CMP2_TZQZ','CCCSDT_TZQZ','deltacc']]
# df.columns = ['MP2 DZ','MP2_DZ_Error','MP2 CBS(TZ/QZ)']
df = df.round().astype(int)
df.index += 1

df.to_clipboard(index=True, excel=True,sep=',')
df

,LMP2_TZQZ,LCCSDT_TZQZ,deltacc_local,CMP2_TZQZ,CCCSDT_TZQZ,deltacc
1,-153,-160,-7,-153,-160,-7
2,-183,-190,-7,0,0,0
3,-185,-194,-9,0,0,0


In [56]:
print(ene_extrap_list)

{'MP2_DZ': [0, 0, -155.5188173753127, -173.91585432069638, -164.11473287726284, -156.58078082561974, -157.92722702008055, -159.29369042969194], 'MP2_TZ': [], 'MP2_QZ': [], 'MP2_TZQZ': [0, 0, -193.93711460988152, -201.60602565697087, -200.4288269379048], 'MP2_DZTZ': [0, 0, -255.56772134452115, -255.6332251985666, -220.3259174295968], 'MP2_DZTZ_lc': [0, 0, -198.6287471313573, -213.11635028438872, -186.79559095893984], 'MP2_DZTZ_lc_NOCP': [0, 0, -406.342545102635, -369.2958176667102, -350.45649640510254], 'LCCSDT_TZQZ': [], 'LMP2_TZQZ': [], 'CCCSDT_TZQZ': [], 'CMP2_TZQZ': []}


In [65]:
# Computing th
# Ideal
print(ene_list['MP2_DZTZ'][4])


# Large frozen core
print(ene_list['MP2_DZTZ_lc'][4])


# No BSSE correction
print(ene_list['MP2_DZTZ_lc_NOCP'][4])

# Computing energy at the smaller cluster level

ene_hf = []
ene_mp2 = []
for k in ['TZ','QZ']:
    ene_hf += [get_eads('Data/MP2_Convergence/{0}/{1}'.format(0,k), code_format='mrcc',typ='hf')*Hartree*1000]
    ene_mp2 += [get_eads('Data/MP2_Convergence/{0}/{1}'.format(0,k), code_format='mrcc',typ='lmp2_corr')*Hartree*1000]
dummy = extrapolate.get_cbs(ene_hf[0],ene_mp2[0],ene_hf[1],ene_mp2[1],X=3,Y=4,family='mixcc',output=False)
print(dummy[-1])



-195.99702647487015
-164.65672576997721
-293.9386099745158
-183.91804505946058


-293.9386099745158

[2.413740852989097, 2.441022358367883, 2.495257812255822, 2.40715569211107, 2.4764534646784124, 2.5040234062149587]


In [41]:
# Effects of layer convergence and other things on the adsorption energy

for i in [1,2,3,4]:
    ad_slab = find_energy('Data/DFT/Convergence/{0}/AD_SLAB/OUTCAR'.format(i),code_format='vasp')
    ad =  find_energy('Data/DFT/Convergence/{0}/AD/OUTCAR'.format(i),code_format='vasp')
    slab = find_energy('Data/DFT/Convergence/{0}/SLAB/OUTCAR'.format(i),code_format='vasp')

    eads = (ad_slab-ad-slab)*1000

    print(eads)


-206.40847999999323
-203.08459000000312
-207.28094999992663
-205.52297000000408


In [3]:
params = [61,1] # [ Temperature, degrees of freedom = 3N-5 ] Temperature given as average between Dohnalek and Wichtendahl

thermal_ene_contributions = {y: {x:0 for x in dft_functionals} for y in ['ZPE','Eth','RT','Ecor']}

for i in dft_functionals:
    a,b = read_vib_freq('Data/DFT/Vib_Energy/{0}/AD_SLAB/OUTCAR'.format(i))
    c,d = read_vib_freq('Data/DFT/Vib_Energy/{0}/AD/OUTCAR'.format(i))

    ene_ad,eth_ad,zpe_ad,kT = get_quasi_rrho(a,b,params[0])
    ene_ad_slab,eth_ad_slab,zpe_ad_slab,kT = get_quasi_rrho(c,d,params[0])
    thermal_ene_contributions['ZPE'][i] = zpe_ad_slab - zpe_ad
    thermal_ene_contributions['Eth'][i] = eth_ad_slab - eth_ad
    thermal_ene_contributions['Ecor'][i] = ene_ad-ene_ad_slab + kT
    thermal_ene_contributions['RT'][i] = kT

df = pd.DataFrame(thermal_ene_contributions)
df = df.round().astype(int)
df.to_clipboard(index=True, excel=True,sep=',')
df

,ZPE,Eth,RT,Ecor
01_PBE-D2-Ne,-31,6,5,30
02_revPBE-D4,-28,5,5,28
03_vdW-DF,-28,5,5,28
04_rev-vdW-DF2,-32,6,5,31
05_PBE0-D4,-30,5,5,30
06_B3LYP-D2-Ne,-29,5,5,29


In [5]:
thermal_ene_contributions

{'ZPE': {'01_PBE-D2-Ne': -31.014676535264385,
  '02_revPBE-D4': -28.248867492644933,
  '03_vdW-DF': -27.85794229893733,
  '04_rev-vdW-DF2': -32.2089181470802,
  '05_PBE0-D4': -29.893059328411226,
  '06_B3LYP-D2-Ne': -28.6556890222694},
 'Eth': {'01_PBE-D2-Ne': 5.830647014620704,
  '02_revPBE-D4': 5.356371629189847,
  '03_vdW-DF': 5.43197366139566,
  '04_rev-vdW-DF2': 6.090160539761481,
  '05_PBE0-D4': 5.448381682669231,
  '06_B3LYP-D2-Ne': 5.276644065108683},
 'RT': {'01_PBE-D2-Ne': 5.2565715057024995,
  '02_revPBE-D4': 5.2565715057024995,
  '03_vdW-DF': 5.2565715057024995,
  '04_rev-vdW-DF2': 5.2565715057024995,
  '05_PBE0-D4': 5.2565715057024995,
  '06_B3LYP-D2-Ne': 5.2565715057024995},
 'Total': {'01_PBE-D2-Ne': -25.184029520643662,
  '02_revPBE-D4': -22.892495863455082,
  '03_vdW-DF': -22.4259686375417,
  '04_rev-vdW-DF2': -26.118757607318713,
  '05_PBE0-D4': -24.44467764574199,
  '06_B3LYP-D2-Ne': -23.3790449571608}}

In [3]:
print(ene_ad,eth_ad,zpe_ad)
print(ene_ad_slab,eth_ad_slab,zpe_ad_slab)
print(ene_ad_slab-ene_ad,eth_ad_slab-eth_ad,zpe_ad_slab-zpe_ad,kT)

174.3536150830741 7.88835895460197 166.4652561284721
150.9745701259133 13.165003019710653 137.8095671062027
-23.3790449571608 5.276644065108683 -28.6556890222694 5.2565715057024995
